In [ ]:
%pip install langchain openai transformers --quiet
%pip install -U langchain-openai --quiet
%pip install torch torchvision torchaudio --quiet
%pip install -U langchain-community --quiet
%pip install ipywidgets --quiet
%pip install pandas --quiet
%pip install scikit-learn --quiet

In [ ]:
import os
import pandas as pd
from PIL import Image as PILImage
from PIL import Image, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
import torch
from langchain.agents import initialize_agent
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.tools import BaseTool
from sentence_transformers import SentenceTransformer, util
import chromadb
import base64
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
from IPython.display import Image, display, Markdown
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-FjdZMT6JmIaY0se1k5l8T3BlbkFJsqP2vb3sS2B3zDGlnOeG"

# Initialise ChatGPT
gpt4o = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Initialise CLIP model and processor
clip_model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)
clip_model = CLIPModel.from_pretrained(clip_model_name)

# Initialise the ChromaDB client
chroma_client = chromadb.PersistentClient(path="./image_vdb")

# Initialise ChromaDB Image Loader
image_loader = ImageLoader()
image_loader._PILImage = PILImage

# Initialize CLIP embedding
CLIP = OpenCLIPEmbeddingFunction()
# Create a database of image vectors
image_vdb = chroma_client.get_or_create_collection(name="image", embedding_function=CLIP, data_loader=image_loader)

In [ ]:
# Image Preprocessing Functions
def preprocess_image(image_path):
    image = PILImage.open(image_path).convert('RGB')

    # Resizing images
    image = image.resize((224, 224))

    # Enhanced contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)

    # Denoising
    image = image.filter(ImageFilter.MedianFilter(size=3))

    return image

def extract_image_features(image_path, text):
    # Pre-processed images
    image_object = preprocess_image(image_path)

    # Image feature extraction using CLIP model
    clip_image_inputs = clip_processor(images=image_object, return_tensors="pt")
    clip_text_inputs = clip_processor(text=[text], return_tensors="pt")

    with torch.no_grad():
        clip_image_outputs = clip_model.get_image_features(**clip_image_inputs)
        clip_text_outputs = clip_model.get_text_features(**clip_text_inputs)

    # Normalize the features to have unit norm
    clip_image_outputs = torch.nn.functional.normalize(clip_image_outputs, p=2, dim=-1)
    clip_text_outputs = torch.nn.functional.normalize(clip_text_outputs, p=2, dim=-1)

    # Combine image and text features
    combined_features = (clip_image_outputs + clip_text_outputs) / 2

    # Spreading feature vectors into a one-dimensional list
    image_features = combined_features.flatten().tolist()

    return image_features


In [ ]:
def clear_vdb():
    # Delete the existing collection
    chroma_client.delete_collection(name="image")

    # Recreate the collection
    global image_vdb
    image_vdb = chroma_client.get_or_create_collection(
        name="image", 
        embedding_function=CLIP, 
        data_loader=image_loader
    )

def add_images_to_vdb(directory_paths, max_images_per_folder=120):
    ids = []
    uris = []
    embeddings = [] 
    global_id = 0

    for directory_path in directory_paths:
        images_processed = 0
        for filename in sorted(os.listdir(directory_path)):
            if filename.lower().endswith('.jpg'):
                file_path = os.path.join(directory_path, filename)
                text = os.path.basename(os.path.dirname(file_path))
                print(f"Extracted text for {file_path}: {text}")
                image_features = extract_image_features(file_path, text)
                
                # Add this line to print the first 10 elements of the image features
                print(f"First 10 features for {file_path}: {image_features[:10]}")
                
                ids.append(str(global_id))
                uris.append(file_path)
                embeddings.append(image_features)
                global_id += 1
                images_processed += 1

                # Maximum number of images per folder max_images_per_folder
                if images_processed >= max_images_per_folder:
                    break

    # Add all images to the database at once
    if ids:
        image_vdb.add(ids=ids, uris=uris, embeddings=embeddings) 
        print(f"Added {len(ids)} images to the database.")

# Define the local image catalogue
local_image_dirs = [
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P0\Fabricated',
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P0\Forged',
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P0\Other',
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P0\Sheet Metal',
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P0\Turned'
]

# Clear the vector database before adding new images
clear_vdb()

# Call function to add image to vector database
add_images_to_vdb(local_image_dirs)
print("Image count in VDB:", image_vdb.count())

In [ ]:
def query_db(image_path, text, results=1, top_k=119):
    query_features = extract_image_features(image_path, text)
    try:
        all_results = image_vdb.query(
            query_embeddings=[query_features],
            n_results=top_k + 1,
            include=['uris', 'distances']
        )
    except Exception as e:
        print(f"Error querying the database: {e}")
        return None
    
    filtered_results = filter_results(all_results, image_path, top_k=results)
    return filtered_results

def filter_results(all_results, image_path, top_k):
    distances_and_uris_and_ids = list(zip(all_results['distances'][0], all_results['uris'][0], all_results['ids'][0]))
    filtered_results = [
        (dist, uri, id) for dist, uri, id in distances_and_uris_and_ids
        if os.path.normpath(uri) != os.path.normpath(image_path)
    ]
    sorted_results = sorted(filtered_results, key=lambda x: x[0])[:top_k]
    sorted_uris = [uri for _, uri, _ in sorted_results]
    sorted_distances = [dist for dist, _, _ in sorted_results]
    sorted_ids = [id for _, _, id in sorted_results]
    return {
        'ids': [sorted_ids],
        'uris': [sorted_uris],
        'distances': [sorted_distances]
    }

def format_prompt_inputs(data, user_query, text_info):
    inputs = {}
    inputs['user_query'] = user_query
    inputs['text_info'] = text_info

    image_path_1 = data['uris'][0][0]
    with open(image_path_1, 'rb') as image_file:
        image_data_1 = image_file.read()
    inputs['image_data_1'] = base64.b64encode(image_data_1).decode('utf-8')

    retrieved_texts = [os.path.basename(os.path.dirname(uri)) for uri in data['uris'][0]]
    inputs['retrieved_info_texts'] = retrieved_texts[0] if retrieved_texts else ''

    if data['uris'][0]:
        with open(data['uris'][0][0], 'rb') as image_file:
            inputs['retrieved_info_images_0'] = base64.b64encode(image_file.read()).decode('utf-8')
    else:
        inputs['retrieved_info_images_0'] = ''

    inputs['retrieved_info'] = {
        'uris': data['uris'][0],
        'distances': data['distances'][0],
        'texts': retrieved_texts,
    }

    return inputs

In [ ]:
# Initialise the output parser
parser = StrOutputParser()

image_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a mechanical engineer specializing in manufacturing processes. Your task is to analyze images of mechanical components, identify their primary manufacturing method, and briefly describe key features of the component.

    You will be provided with two images:
    1. The main image to analyze, which is a top view of a mechanical component.
    2. A similar component retrieved from the database, which is an isometric view.

    Your goal is to use both images for comparison and to enhance your analysis, providing a more comprehensive and accurate description."""),

    ("user", [
        {"type": "text", "text": """Analyze the main image of the mechanical component and identify its primary manufacturing method and key features.

You have access one similar image retrieved from the database, the retrieved image is displayed below.

Consider the similarities and differences between the main image and the retrieved image to provide a more accurate and comprehensive description.

Respond with a concise sentence stating the primary manufacturing method used to produce this component and key features."""},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_1}"},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{retrieved_info_images_0}"},
    ]),
])

# Define the LangChain chain
vision_chain = image_prompt | gpt4o | parser

def generate_description_with_rag(image_path):
    try:
        text_info = os.path.basename(os.path.dirname(image_path))
        # Setting up problems directly within a function
        user_query = "Analyze the mechanical component image with reference to similar components."
        
        results = query_db(image_path, text_info, results=1)
        print(f"Query results for {image_path}: {results}")
        
        if results is None or len(results['uris'][0]) == 0:
            print(f"Warning: No results retrieved for {image_path}")
            return "Unable to answer the question due to lack of similar components."
        
        prompt_input = format_prompt_inputs(results, user_query, text_info)
        
        response = vision_chain.invoke(prompt_input)
        return response
    except Exception as e:
        print(f"Error in generate_description_with_rag for {image_path}: {str(e)}")
        return f"Error answering the question: {str(e)}"

In [ ]:
# Define a tool class for describing images
class DescribeImageWithDetailsTool(BaseTool):
    name = "Describe Image With Details Tool"
    description = 'Use this tool to describe an images of mechanical parts with additional details.'
    
    def _run(self, image_path: str):
        return self.run(image_path)
    
    def run(self, image_path: str):
        description = generate_description_with_rag(image_path)
        return description

# Initialise tools and agents
tools = [DescribeImageWithDetailsTool()]
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=gpt4o,
    max_iterations=5,
    early_stopping_method='generate'
)

In [ ]:
def process_directories(directory_paths):
    results = []
    for directory_path in directory_paths:
        missing_files = []
        files_processed = 0
        for filename in os.listdir(directory_path):
            if filename.lower().endswith('.jpg'):
                files_processed += 1
                if files_processed > 1:
                    break
                image_path = os.path.join(directory_path, filename)
                print(f"Checking if the file exists: {image_path}")
                if os.path.exists(image_path):
                    try:
                        print(f"Calling agent to process the file: {image_path}")
                        description = generate_description_with_rag(image_path)
                        result = {"image_name": filename, "description": description}
                        results.append(result)
                    except Exception as e:
                        results.append({"image_name": filename, "description": f"Error processing file: {str(e)}"})
                else:
                    print(f"File not found: {image_path}")
                    missing_files.append(filename)
                    results.append({"image_name": filename, "description": "File not found"})
        if missing_files:
            print(f"The following files were not found in {directory_path}: {', '.join(missing_files)}")

    # Save results to a CSV file
    output_file_path = os.path.join(os.getcwd(), "RAG_try.csv")
    df = pd.DataFrame(results)
    df.to_csv(output_file_path, index=False)
    print(f"CSV file saved at: {output_file_path}")

    return results
 
description_dirs = [
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Fabricated',
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Forged',
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Other',
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Sheet Metal',
    r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Turned'
]


# Process the catalogue and generate a description
results = process_directories(description_dirs)

# Show results
for result in results:
    print(f"{result['image_name']}: {result['description']}")